In [1]:
import keras_hub
import random
import requests

import keras
from keras import ops

import tensorflow.data as tf_data
import tensorflow as tf
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset,
)
import pandas as pd
from keras_nlp.samplers import TopKSampler
from sklearn.metrics import f1_score


import numpy as np
import os
from pathlib import Path

/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for pre-parsing
from pyparsing import Word, alphas as pp_alpha, nums as pp_nums
import pyparsing as pp
pp.ParserElement.enablePackrat()
import re

In [ ]:
class DictTokenizer:
    def __init__(self, vocab, tokenizer_fn):
        self.token_to_id_map = vocab
        self.id_to_token_map = {i: t for t, i in vocab.items()}
        self.tokenizer_fn = tokenizer_fn

    def __call__(self, text_batch):
        return [
            [self.token_to_id_map.get(tok, self.token_to_id_map.get("[UNK]", 0)) 
             for tok in self.tokenizer_fn(text)]
            for text in text_batch
        ]

    def tokenize(self, text):
        return [self.token_to_id_map.get(tok, self.token_to_id_map.get("[UNK]", 0)) 
                for tok in self.tokenizer_fn(text)]

    def detokenize(self, token_ids):
        if isinstance(token_ids, tf.Tensor):
            token_ids = token_ids.numpy()
        elif isinstance(token_ids, tf.RaggedTensor):
            token_ids = token_ids.to_tensor().numpy()
        elif isinstance(token_ids, int):
            token_ids = [token_ids]

        return " ".join([self.id_to_token_map.get(int(tok_id), "[UNK]") for tok_id in token_ids])

    def token_to_id(self, token):
        return self.token_to_id_map.get(token, self.token_to_id_map.get("[UNK]", 0))


In [4]:
# regex rules

alpha_regexp = r"""
(?!((?:THUMB-)?(?:IX|POSS|SELF)))   # negative lookahead for blocked glosses
[A-Z]                               # must start with uppercase
(?:                                 # optional middle section
    (?:                             # non-capturing group for allowed connectors
        (?:[-/][A-Z])               # hyphen or slash must be followed by uppercase
      | (?:_[0-9])                  # underscore must be followed by digit
      | (?:\+(?:[A-Z#]|fs-))       # plus + (uppercase OR # OR the literal fs-)
      | [A-Z0-9]                    # regular letter/digit continuation
    )
)*                                  # repeatable
(?:\.)?                             # optional trailing period
"""

In [5]:
# conventions kept for parsing

cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"])
fs_prefix = pp.Literal("fs-")
index_core_ix = pp.Literal("IX")
other_index_core = pp.one_of(["POSS", "SELF"])
hashtag = pp.Literal("#")
dash = pp.Literal("-")
contraction = pp.Literal("^")
period = pp.Literal(".")
alpha = pp.Word(pp_alpha, max=1)
num = pp.Word(pp_nums, max=1)
word = pp.Regex(alpha_regexp, flags=re.X)

In [6]:
# grammar rules

full_grammar = pp.OneOrMore(
    fs_prefix |               # fingerspelling fs
    word |
    cl_prefix |               # classifiers like CL, DCL, etc.
    index_core_ix |           # IX
    other_index_core |        # POSS, SELF
    hashtag |                 # #
    contraction |             # ^
    period |                  # .
    dash |
    num |
    alpha                     # fallback LAST
)

In [7]:
# tokenize based on predefined grammar rules

def custom_asl_tokenize(text):
    try:
        if "'" in text:
            text = text.replace("'", "")
        if "++" in text:
            text = text.replace("++", "+")
        return full_grammar.parse_string(text, parse_all=True).asList()
    except pp.ParseException as pe:
        print(text)
        print(f"Failed to parse: {pe}")
        return []

In [8]:
def custom_eng_tokenize(text):
    # Perserve punctuation and digits
    text = re.sub(r'([^\w\s]|\d)', r' \1 ', text)
    # Convert to lowercase
    text = text.lower()
    # Split on whitespace
    tokens = text.split()
    return tokens

In [9]:
# generate
    # 1) list of eng-asl sentence pairs
    # 2) set of unique english vocab
    # 3) set of unique asl vocab
data_path = "/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/sent_pairs_joined.txt"
    
text_pairs = []
eng_texts = []
asl_texts = []
SPECIAL_TOKENS = ["[PAD]", "[START]", "[END]", "[UNK]"]
eng_tokens = set(SPECIAL_TOKENS)
asl_tokens = set(SPECIAL_TOKENS)
max_length = 0

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines:
    pair = []
    eng_text, asl_text = line.split("\t")
    eng_texts.append(eng_text)
    asl_texts.append(asl_text)
    pair.append(eng_text.lower())
    pair.append(asl_text)
    text_pairs.append(pair)
    
for text in eng_texts:
    tokens = custom_eng_tokenize(text)
    length = len(tokens)
    if length > max_length:
        max_length = length
    for token in tokens:
        if token not in eng_tokens:
                eng_tokens.add(token)
            
for text in asl_texts:
    tokens = custom_asl_tokenize(text)
    length = len(tokens)
    if length > max_length:
        max_length = length
    for token in tokens:
        if token not in asl_tokens:
                asl_tokens.add(token)
                            
max_encoder_seq_length = max([len(txt) for txt in eng_texts])
max_decoder_seq_length = max([len(txt) for txt in asl_texts])

eng_tokens = sorted(list(eng_tokens))
asl_tokens = sorted(list(asl_tokens))

print("eng_tokens:", eng_tokens)
print("asl_tokens", asl_tokens)
num_encoder_tokens = len(eng_tokens)
num_decoder_tokens = len(asl_tokens)
print("num_eng_tokens", num_encoder_tokens)
print("num_asl_tokens", num_decoder_tokens)

eng_tokens: ['!', '"', '$', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[END]', '[PAD]', '[START]', '[UNK]', '`', 'a', 'aaa', 'aaahhh', 'abcs', 'abdominal', 'able', 'about', 'above', 'abuse', 'accept', 'acceptable', 'acceptance', 'accepted', 'accident', 'accidentally', 'accommodations', 'accomplishment', 'across', 'act', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actually', 'added', 'addicted', 'addicts', 'admit', 'adopt', 'adopted', 'adopts', 'advantage', 'adventure', 'advice', 'advisior', 'ae', 'afford', 'afraid', 'after', 'afternoon', 'again', 'against', 'age', 'ago', 'agree', 'agreed', 'ah', 'ahead', 'ahh', 'ail', 'air', 'airline', 'airlines', 'airplanes', 'alarm', 'alcohol', 'alec', 'ali', 'alike', 'alive', 'all', 'allergic', 'allow', 'allowed', 'almost', 'alone', 'along', 'alphabetical', 'already', 'alright', 'also', 'always', 'am', 'amazing', 'american', 'amy', 'an', 'analyze', 'analyzed', 'analy

In [10]:
main_asl_glosses = set()
split_pattern = r"[\/\+\-]"
        
for token in asl_tokens:
    parts = re.split(split_pattern, token)
    if all(part.isalpha() and part.isupper() for part in parts):
        main_asl_glosses.add(token)

In [11]:
# model parameters / hyperparameters

BATCH_SIZE = 16
EPOCHS = 20
EMBED_DIM = 64
INTERMEDIATE_DIM = 128
NUM_HEADS = 4
MAX_SEQUENCE_LENGTH = max_length

In [12]:
# glimpse pairs

for _ in range(5):
    print(random.choice(text_pairs))

["no, i don't have a car.", 'IX NONE/NOTHING CAR']
["i'm letting you know that the car is broken down, i used the the right key.", 'INFORM IX CAR BREAK-DOWN IX USE_2 CORRECT KEY']
['how many kids does phil have?', 'fs-PHIL HAVE KID CHILDREN HOW-MANY']
["no, the car didn't breakdown.", 'CAR BREAK-DOWN #NO']
['one friend asked: "why is your mom angry?", "my mom angry?" asked the coda child. who then looked over at...', 'LOOK ATTENTION-WAVE WHY POSS MOTHER ANGRY LOOK MOTHER ANGRY LOOK']


In [13]:
# split data

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

3389 total pairs
2373 training pairs
508 validation pairs
508 test pairs


In [14]:

eng_vocab = dict([(char, i) for i, char in enumerate(eng_tokens)])
asl_vocab = dict([(char, i) for i, char in enumerate(asl_tokens)])

eng_tokenizer = DictTokenizer(eng_vocab, tokenizer_fn=custom_eng_tokenize)
asl_tokenizer = DictTokenizer(asl_vocab, tokenizer_fn=custom_asl_tokenize)

print(eng_tokenizer)
print(asl_tokenizer)

print(eng_vocab)
print(asl_vocab)

{'!': 0, '"': 1, '$': 2, "'": 3, '(': 4, ')': 5, '*': 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, '[END]': 24, '[PAD]': 25, '[START]': 26, '[UNK]': 27, '`': 28, 'a': 29, 'aaa': 30, 'aaahhh': 31, 'abcs': 32, 'abdominal': 33, 'able': 34, 'about': 35, 'above': 36, 'abuse': 37, 'accept': 38, 'acceptable': 39, 'acceptance': 40, 'accepted': 41, 'accident': 42, 'accidentally': 43, 'accommodations': 44, 'accomplishment': 45, 'across': 46, 'act': 47, 'acting': 48, 'action': 49, 'actions': 50, 'activities': 51, 'actor': 52, 'actors': 53, 'actually': 54, 'added': 55, 'addicted': 56, 'addicts': 57, 'admit': 58, 'adopt': 59, 'adopted': 60, 'adopts': 61, 'advantage': 62, 'adventure': 63, 'advice': 64, 'advisior': 65, 'ae': 66, 'afford': 67, 'afraid': 68, 'after': 69, 'afternoon': 70, 'again': 71, 'against': 72, 'age': 73, 'ago': 74, 'agree': 75, 'agreed': 76, 'ah': 77, 'ahead': 78, 'ahh': 79,

In [15]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

asl_input_ex = text_pairs[0][1]
asl_tokens_ex = asl_tokenizer.tokenize(asl_input_ex)
print("ASL sentence: ", asl_input_ex)
print("Tokens: ", asl_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    asl_tokenizer.detokenize(asl_tokens_ex),
)

English sentence:  i was driving for a long time.
Tokens:  [1324, 2955, 816, 1051, 29, 1599, 2769, 9]
Recovered text after detokenizing:  i was driving for a long time .

ASL sentence:  IX DRIVE
Tokens:  [950, 524]
Recovered text after detokenizing:  IX DRIVE


In [16]:
def preprocess_batch(eng, asl):
    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
        dtype="int32"
    )
    eng = eng_start_end_packer(eng)

    asl_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=asl_tokenizer.token_to_id("[START]"),
        end_value=asl_tokenizer.token_to_id("[END]"),
        pad_value=asl_tokenizer.token_to_id("[PAD]"),
        dtype="int32"
    )
    asl = asl_start_end_packer(asl)

    decoder_inputs = asl[:, :-1]
    decoder_outputs = asl[:, 1:]

    return {
        "encoder_inputs": eng,
        "decoder_inputs": decoder_inputs
    }, decoder_outputs


In [17]:
def make_dataset(pairs):
    
    eng_ids = [eng_tokenizer.tokenize(sent) for sent, _ in pairs]    
    asl_ids = [asl_tokenizer.tokenize(sent) for _, sent in pairs]

    # 🛠️ Force token type to int32
    eng_tensor = tf.ragged.constant(eng_ids, dtype=tf.int32)
    asl_tensor = tf.ragged.constant(asl_ids, dtype=tf.int32)
    
    dataset = tf_data.Dataset.from_tensor_slices((eng_tensor, asl_tensor))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
print(train_ds)

2025-07-20 12:57:50.093394: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-07-20 12:57:50.093704: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-07-20 12:57:50.093726: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1753030670.094310 15256134 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1753030670.094403 15256134 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<CacheDataset element_spec=({'encoder_inputs': TensorSpec(shape=(None, 71), dtype=tf.int32, name=None), 'decoder_inputs': TensorSpec(shape=(None, 71), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 71), dtype=tf.int32, name=None))>


In [18]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (16, 71)
inputs["decoder_inputs"].shape: (16, 71)
targets.shape: (16, 71)


2025-07-20 12:57:52.189762: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-07-20 12:57:52.190509: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:

# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=num_encoder_tokens,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=num_decoder_tokens,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

transformer.summary()
transformer.compile(
    optimizer, loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 64)  │    202,688 │ encoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 64)  │     33,472 │ token_and_positi… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, None,      │    329,683 │ decoder_inputs[0… │
│ (Functional)        │ 2131)             │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 565,843 (2.16 MB)

 Trainable params: 565,843 (2.16 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


2025-07-20 12:57:58.769655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


149/149 ━━━━━━━━━━━━━━━━━━━━ 28s 134ms/step - loss: 6.1947 - sparse_categorical_accuracy: 0.6563 - val_loss: 4.4505 - val_sparse_categorical_accuracy: 0.8648
Epoch 2/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 23s 154ms/step - loss: 4.0172 - sparse_categorical_accuracy: 0.8657 - val_loss: 2.6775 - val_sparse_categorical_accuracy: 0.8648
Epoch 3/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - loss: 2.4026 - sparse_categorical_accuracy: 0.8657 - val_loss: 1.5403 - val_sparse_categorical_accuracy: 0.8648
Epoch 4/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - loss: 1.4961 - sparse_categorical_accuracy: 0.8658 - val_loss: 1.0832 - val_sparse_categorical_accuracy: 0.8650
Epoch 5/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - loss: 1.1275 - sparse_categorical_accuracy: 0.8690 - val_loss: 0.9158 - val_sparse_categorical_accuracy: 0.8753
Epoch 6/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - loss: 0.9669 - sparse_categorical_accuracy: 0.8767 - val_loss: 0.8411 - val_sparse_categorical_accuracy: 0.8834


In [ ]:
class VocabSampler(keras_hub.samplers.Sampler):
    def __init__(self, sentence, vocab,
                 **kwargs):
        super().__init__(**kwargs)
        self.input_words    = [w for w in sentence if len(w) > 2]
        self.vocab          = vocab
        self.prefered_boost = 2.0
        self.allowed_boost  = 1.5
        self.temperature    = 0.8
        self.top_k          = 10               # store it
        self.max_repeats    = 2
        # precompute once
        self.allowed_ids    = self.compute_allowed_ids("allowed")
        self.prefered_ids   = self.compute_allowed_ids("prefer")
        self.prev_token     = None
        self.repeat_count   = 0

    def compute_allowed_ids(self, preference):
        prefer, allowed = set(), set()
        for word in self.input_words:
            p, s = word[:3], word[-3:]
            for gloss, gid in self.vocab.items():
                if gloss.startswith(p) and gloss.endswith(s):
                    prefer.add(gid)
                elif gloss.startswith(p) or gloss.endswith(s):
                    allowed.add(gid)
        return prefer if preference=="prefer" else allowed

    def get_next_token(self, logits):
        # 1) Boost matching gloss logits
        for gid in self.allowed_ids:
            boosted = logits[0, gid] + self.allowed_boost
            logits  = keras.ops.slice_update(
                logits, (0, gid),
                keras.ops.reshape(boosted, (1,1))
            )
        for gid in self.prefered_ids:
            boosted = logits[0, gid] + self.prefered_boost
            logits  = keras.ops.slice_update(
                logits, (0, gid),
                keras.ops.reshape(boosted, (1,1))
            )

        # 2) Repetition penalty
        if self.prev_token is not None and self.repeat_count >= self.max_repeats:
            neg_inf = tf.constant([[-1e9]], dtype=logits.dtype)
            logits  = keras.ops.slice_update(logits, (0, self.prev_token), neg_inf)

        # 3) Temperature scaling
        scaled_logits = logits / tf.cast(self.temperature, logits.dtype)

        # 4) Top‑k filtering via threshold
        topk_vals, _ = tf.math.top_k(scaled_logits, k=self.top_k)
        threshold    = tf.reduce_min(topk_vals, axis=-1, keepdims=True)  # [batch,1]
        neg_inf      = tf.constant(-1e9, dtype=scaled_logits.dtype)
        filtered     = tf.where(
            scaled_logits < threshold,
            neg_inf,
            scaled_logits
        )

        # 5) Sample from the filtered logits
        # tf.random.categorical expects 2D [batch, vocab_size]
        sample_id = tf.random.categorical(filtered, num_samples=1)  # [batch,1]
        token     = tf.squeeze(sample_id, axis=1)                  # [batch]

        # 6) Update repeat tracking
        tid = int(token.numpy()[0])
        if tid == self.prev_token:
            self.repeat_count += 1
        else:
            self.prev_token   = tid
            self.repeat_count = 1

        return token  # shape [1]


In [41]:
class BabelNetSampler(keras_hub.samplers.Sampler):
    def __init__(self, sentence, vocab,
                 key = "70426f32-2291-4d4e-9f6d-a220bc1a93f6",
                 lang = "EN",
                 target_lan = "EN",
                 id_url = "https://babelnet.io/v9/getSynsetIds",
                 word_url = "https://babelnet.io/v9/getSenses",
                 **kwargs):
        super().__init__(**kwargs)
        self.input_words    = [w for w in sentence if len(w) > 2]
        self.vocab          = vocab
        self.prefered_boost = 2.0
        self.allowed_boost  = 1.5
        self.temperature    = 0.8
        self.top_k          = 10               # store it
        self.max_repeats    = 2
        self.id_url = id_url
        self.word_url = word_url
        self.key = key
        self.target = target_lan
        self.lang = lang
        # precompute once
        self.allowed_ids    = self.compute_allowed_ids("allowed")
        self.prefered_ids   = self.compute_allowed_ids("prefer")
        self.prev_token     = None
        self.repeat_count   = 0
    
    def fetch_senses(self, word):
        word = word.lower()
        header = {'Accept-Encoding':'gzip'}
        word_params = {"lemma": word, "searchLang": self.lang, "targetLang":self.target, "key": self.key}
        candidates = set()
        try:
            resp = requests.get(self.word_url, params=word_params, headers=header)
            resp.raise_for_status()
            data = resp.json()
            for each in data:
                if each.get("type") == "BabelSense":
                    props = each.get("properties", {})
                    lemmas = props.get("lemma", {})
                    lemma = lemmas.get("lemma")                    
                    if lemma.isalpha() and (lemma.upper() not in candidates):
                        candidates.add(lemma.upper())
        except Exception as e:
            print(f"BabelNet fetch error for '{word}':", e)
            raise
        
        return candidates
            

    def compute_allowed_ids(self, preference):
        prefer, allowed = set(), set()
        for word in self.input_words:
            candidates = self.fetch_senses(word)
            print(candidates)
            for each in candidates:
                p, s = each[:3], each[-3:]
                print(p)
                print(s)
                for gloss, gid in self.vocab.items():
                    if gloss.startswith(p) and gloss.endswith(s):
                        prefer.add(gid)
                    elif gloss.startswith(p) or gloss.endswith(s):
                        allowed.add(gid)
        print(prefer)
        print(allowed)
        return prefer if preference=="prefer" else allowed

    def get_next_token(self, logits):
        # 1) Boost matching gloss logits
        for gid in self.allowed_ids:
            print(gid)
            boosted = logits[0, gid] + self.allowed_boost
            logits  = keras.ops.slice_update(
                logits, (0, gid),
                keras.ops.reshape(boosted, (1,1))
            )
        for gid in self.prefered_ids:
            print(gid)
            boosted = logits[0, gid] + self.prefered_boost
            logits  = keras.ops.slice_update(
                logits, (0, gid),
                keras.ops.reshape(boosted, (1,1))
            )

        # 2) Repetition penalty
        if self.prev_token is not None and self.repeat_count >= self.max_repeats:
            neg_inf = tf.constant([[-1e9]], dtype=logits.dtype)
            logits  = keras.ops.slice_update(logits, (0, self.prev_token), neg_inf)

        # 3) Temperature scaling
        scaled_logits = logits / tf.cast(self.temperature, logits.dtype)

        # 4) Top‑k filtering via threshold
        topk_vals, _ = tf.math.top_k(scaled_logits, k=self.top_k)
        threshold    = tf.reduce_min(topk_vals, axis=-1, keepdims=True)  # [batch,1]
        neg_inf      = tf.constant(-1e9, dtype=scaled_logits.dtype)
        filtered     = tf.where(
            scaled_logits < threshold,
            neg_inf,
            scaled_logits
        )

        # 5) Sample from the filtered logits
        # tf.random.categorical expects 2D [batch, vocab_size]
        sample_id = tf.random.categorical(filtered, num_samples=1)  # [batch,1]
        token     = tf.squeeze(sample_id, axis=1)                  # [batch]

        # 6) Update repeat tracking
        tid = int(token.numpy()[0])
        if tid == self.prev_token:
            self.repeat_count += 1
        else:
            self.prev_token   = tid
            self.repeat_count = 1

        return token  # shape [1]

In [42]:
def decode_sequences(input_sentences):
    with tf.device('/CPU:0'):
        batch_size     = 1
        prompt_length  = MAX_SEQUENCE_LENGTH   # still pad encoder/prompt to fixed size

        # — Encoder input preparation (unchanged) —
        encoder_inputs = ops.convert_to_tensor(eng_tokenizer(input_sentences))
        seq_len = tf.shape(encoder_inputs)[1]
        if seq_len < prompt_length:
            pad_amt = prompt_length - seq_len
            pads    = ops.full((1, pad_amt), 0, dtype=encoder_inputs.dtype)
            encoder_inputs = ops.concatenate([encoder_inputs, pads], axis=1)

        # — Compute dynamic decode lengths —
        input_ids      = eng_tokenizer(input_sentences)[0]
        input_len      = len(input_ids)
        max_decode_len = min(prompt_length, input_len)               # at most input+2
        min_decode_len = max(1, int(input_len * 0.75))                   # at least 80% of input

        # — Initialize your sampler as before —
        sampler = BabelNetSampler(
            sentence=input_sentences[0].split(),
            vocab=asl_tokenizer.token_to_id_map,
        )

        # — Build initial prompt ([START] + PADs) —
        start_id = asl_tokenizer.token_to_id("[START]")
        pad_id   = asl_tokenizer.token_to_id("[PAD]")
        prompt   = ops.full((batch_size, prompt_length), pad_id, dtype=tf.int32)
        start_t  = ops.convert_to_tensor([[start_id]], dtype=tf.int32)
        prompt   = ops.slice_update(prompt, (0, 0), start_t)

        # — Next‑token fn (unchanged) —
        def next_fn(pr, cache, idx):
            logits = transformer([encoder_inputs, pr])[:, idx - 1, :]
            return logits, None, cache

        cache     = None
        generated = []

        # — Decoding loop with dynamic bounds —
        for idx in range(1, max_decode_len):
            logits, _, cache = next_fn(prompt, cache, idx)
            token = sampler.get_next_token(logits)                # shape [1]
            token = ops.cast(token, dtype=prompt.dtype)
            prompt = ops.slice_update(prompt, (0, idx), ops.expand_dims(token, 0))

            tok_id = int(token.numpy()[0])
            generated.append(tok_id)

            # only stop if we’ve hit [END] *and* run at least min_decode_len steps
            if tok_id == asl_tokenizer.token_to_id("[END]") and idx >= min_decode_len:
                break

        return asl_tokenizer.detokenize(generated)


outputs = []
test_eng_texts = [pair[0] for pair in test_pairs]

for i in range(5):
    output_pairs = []
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequences([input_sentence])
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    output_pairs.append(input_sentence)
    output_pairs.append(translated)
    print(output_pairs)
    outputs.append(output_pairs)

df = pd.DataFrame(outputs, columns=["input sentence", "translation"])
df.to_csv("/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/seq2seq_code/word_level/babelnet_sampler3.txt", index=False)


{'CHARLES', 'STEPHEN', 'WC', 'IOANN', 'MONGERING', 'JOHANNEAN', 'JOHANNES', 'PRIVY', 'HENRY', 'HARRY', 'KARZY', 'IOANNES', 'WHORES', 'CLIENT', 'PROSITUTION', 'WILLIAM', 'BOG', 'URITROTTOIR', 'PELUSHI', 'IOAN', 'MANNII', 'PETER', 'SKANKS', 'YSAAC', 'HOOKING', 'HOE', 'KERBCRAWLING', 'BILL', 'JOHNS', 'WHOREMONGERS', 'KAMELION', 'BASIL', 'JEAN', 'ΙΩΆΝΝΗΣ', 'JANES', 'COMPANION', 'PROSTITUTION', 'LOO', 'HORCWENAN', 'RESTROOMS', 'COREFUSION', 'MARK', 'HOOKERS', 'ETHNUS', 'CAROL', 'GILLIAN', 'NUMATIC', 'WASHROOM', 'IOANNIS', 'LAVATORY', 'IOHN', 'JONH', 'STRUMPETS', 'ASSISTANT', 'KHAZI', 'WHOOR', 'SPIDRON', 'WHOREMASTER', 'HETTY', 'PROSTITUDE', 'WENDY', 'TOILET', 'IONICĂ', 'KERBCRAWL', 'JACK', 'TOILETROOM', 'ヂャン', 'HARLOTRY', 'JOHN', 'PROSTITUTES', 'PROSITUTE', 'QUEAN', 'LUKE', 'JAMES', 'HARLOTS', 'KARZEY', 'BATHROOM', 'ABRAHAM', 'PUNTER', 'WHOREDOM', 'JOANNES', 'TRICK', 'COMPANIONS', 'KATARINA', 'DAVID', 'PROSTETUTION', 'LEWIS', 'JEDIKIAH', 'EDWARD', 'GENDE', 'WHOREMONGERING', 'JANE', 'WHOREMO